In [2]:
import math
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pickle
from ramannoodles import peakidentify
from ramannoodles import shoyu
from ramannoodles import spectrafit

In [11]:
def test_peak_assignment():
    """This function tests the operation of the peak_assignment function in peakidentify.py"""
    #First, generate a testing dataset.
    shoyu_data_dict = pickle.load(open('../raman_spectra/shoyu_data_dict.p', 'rb'))
    compound_1 = shoyu_data_dict['WATER']
    compound_2 = shoyu_data_dict['CARBON MONOXIDE']
    compound_3 = shoyu_data_dict['CARBON DIOXIDE']
    unknown_x, unknown_y = shoyu.combine_spectra(compound_1, compound_2, plot = False)
    unknown_x = np.asarray(unknown_x)
    unknown_y = np.asarray(unknown_y)
    known_compound_list = [compound_1, compound_2, compound_3]
    precision = 0.03
    
    #Various try statements to make sure that bad inputs are handled correctly. 
    try:
        peak_assignment(1, unknown_y, known_compound_list, precision, False)
    except TypeError:
        print("An invalid unknown_x was passed to the function, and it was handled well with a TypeError.")
    
    try:
        peak_assignment(unknown_x, 2, known_compound_list, precision, False)
    except TypeError:
        print("An invalid unknown_y was passed to the function, and it was handled well with a TypeError.")
        
    try:
        peak_assignment(unknown_x, unknown_y, 'string', precision, False)
    except TypeError:
        print("An invalid known_compound_list was passed to the function, and it was handled well with a TypeError.")

    try:
        peak_assignment(unknown_x, unknown_y, [1, 3, 6], precision, False)
    except TypeError:
        print("An invalid element inside known_compound_list was passed to the function, and it was handled well with a TypeError.")
    
    try:
        peak_assignment(unknown_x, unknown_y, known_compound_list, 'precision', False)
    except TypeError:
        print("An invalid precision value was passed to the function, and it was handled well with a TypeError.")
    
    try:
        peak_assignment(1, unknown_y, known_compound_list, precision, 'False')
    except TypeError:
        print("An invalid plot value was passed to the function, and it was handled well with a TypeError.")
    
    

In [12]:
test_peak_assignment

<function __main__.test_peak_assignment()>

In [14]:
def test_compare_unknown_to_known():
    """This function tests the operation of the compare_unknown_to_known function in peakidentify.py"""
    #Build our test dataset. 
    shoyu_data_dict = pickle.load(open('../raman_spectra/shoyu_data_dict.p', 'rb'))
    compound_1 = shoyu_data_dict['WATER']
    compound_2 = shoyu_data_dict['CARBON MONOXIDE']
    compound_3 = shoyu_data_dict['CARBON DIOXIDE']
    unknown_x, unknown_y = shoyu.combine_spectra(compound_1, compound_2, plot = False)
    unknown_x = np.asarray(unknown_x)
    unknown_y = np.asarray(unknown_y)
    known_compound_list = [compound_1, compound_2, compound_3]
    precision = 0.03
    for i in range(len(known_compound_list)):
        known_peaks[i] = spectrafit.compound_report(known_compound_list[i])
    unknown_peaks = spectrafit.data_report(unknown_x, unknown_y)[0]
    
    try:
        compare_unknown_to_known(1, known_peaks, precision)
    except TypeError:
        print("An invalid unknown_peaks value was passed to the function, and was handled correctly.")
    
    try:
        compare_unknown_to_known(unknown_peaks, 'known_peaks', precision)
    except TypeError:
        print("An invalid known_peaks value was passed to the function, and was handled correctly.")
    
    try:
        compare_unknown_to_known(unknown_peaks, known_peaks, 'precision')
    except TypeError:
        print("An invalid precision value was passed to the function, and was handled correctly.")
        
    #After testing for resilience to unexpected inputs, now ensure outputs are performing as expected. 
    

In [15]:
test_compare_unknown_to_known

<function __main__.test_compare_unknown_to_known()>

In [5]:
def test_peak_position_comparisons():
    """This function tests the operation of the peak_position_comparisons function in peakidentify.py"""
    
    

In [6]:
def test_percentage_of_peaks_found():
    """This function tests the operation of the percentage_of_peaks_found function in peakidentify.py"""
    
    

In [7]:
def test_plotting_peak_assignments():
    """This function tests the operation of the peak_assignment function in peakidentify.py"""
    
    